In [1]:
# Jupyter magic extensions that makes our live easier
%load_ext autoreload
%autoreload 2

In [5]:
from src.models.train_pipeline import train_pipeline
from src.models.models import SklearnAdaBoostModel
from src.data.make_dataset import PandasDatasetLoader

dataset_path = 'E:\\Epilepsy challenge\\1110_train\\data\\parquet\\train'

train_pipeline(PandasDatasetLoader(dataset_path),SklearnAdaBoostModel())

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
import os
print(os.getcwd())

os.chdir('..')



c:\Users\rober\Documents\Epilepsy_Challenge\epilepsia_challenge\notebooks


In [4]:
import os
print(os.getcwd())




c:\Users\rober\Documents\Epilepsy_Challenge\epilepsia_challenge
